In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions
import decimal

spark = SparkSession.builder.master("local").appName("app").getOrCreate()

spark.conf.set(
"fs.azure.account.key.adfrcdevstorage.blob.core.windows.net",
"xrbJuOZpG1+WGZk2yJhbNbaIjB+adtF6d8BN+3xLnYMEDtwA+B3fzi44qPaPNpr8OJgzFXehzBQFC4/ZqX6vCQ=="
)

df_8_05 = spark.read.parquet("wasbs://guidedcapstone-2021-08-09t17-05-44-969z@guidedcapstonhdistorage.blob.core.windows.net/HdiNotebooks/trade3/trade_dt={}".format("2020-08-05"))
df_8_06 = spark.read.parquet("wasbs://guidedcapstone-2021-08-09t17-05-44-969z@guidedcapstonhdistorage.blob.core.windows.net/HdiNotebooks/trade3/trade_dt={}".format("2020-08-06"))

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1631552266226_0007,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
df_8_05.show(5)
df_8_06.show(5)

+----------+------+--------+--------------------+------------+------+--------+------+--------+--------------------+
|  trade_dt|Symbol|exchange|            event_tm|event_seq_nb|bid_pr|bid_size|ask_pr|ask_size|    max_arrival_time|
+----------+------+--------+--------------------+------------+------+--------+------+--------+--------------------+
|2020-08-05|  SYMC|    NYSE|2020-08-05 21:52:...|         100|   161|     312|  null|    null|2020-08-05 09:30:...|
|2020-08-05|  SYMC|    NYSE|2020-08-05 19:37:...|          80|   161|     713|  null|    null|2020-08-05 09:30:...|
|2020-08-05|  SYMB|    NYSE|2020-08-05 14:26:...|          40|    37|     370|  null|    null|2020-08-05 09:30:...|
|2020-08-05|  SYMC|    NYSE|2020-08-05 20:48:...|          90|   160|     692|  null|    null|2020-08-05 09:30:...|
|2020-08-05|  SYMA|    NYSE|2020-08-05 10:37:...|          10|    79|     912|  null|    null|2020-08-05 09:30:...|
+----------+------+--------+--------------------+------------+------+---

In [3]:
# Create a temp trade staging table
from pyspark.sql.functions import col, when

new_df = df_8_06.select('*')
new_df = new_df.withColumn('rec_type', when(new_df.exchange=='NYSE','T'))
new_df.createOrReplaceTempView("tmp_trade_moving_avg")
new_df.show(5)

+----------+------+--------+--------------------+------------+------+--------+------+--------+--------------------+--------+
|  trade_dt|Symbol|exchange|            event_tm|event_seq_nb|bid_pr|bid_size|ask_pr|ask_size|    max_arrival_time|rec_type|
+----------+------+--------+--------------------+------------+------+--------+------+--------+--------------------+--------+
|2020-08-06|  SYMA|    NYSE|2020-08-06 22:00:...|         100|    76|     471|  null|    null|2020-08-06 09:30:...|       T|
|2020-08-06|  SYMA|    NYSE|2020-08-06 15:39:...|          50|    76|     744|  null|    null|2020-08-06 09:30:...|       T|
|2020-08-06|  SYMA|    NYSE|2020-08-06 14:27:...|          40|    77|      81|  null|    null|2020-08-06 09:30:...|       T|
|2020-08-06|  SYMB|    NYSE|2020-08-06 21:46:...|         100|    36|     741|  null|    null|2020-08-06 09:30:...|       T|
|2020-08-06|  SYMC|    NYSE|2020-08-06 11:52:...|          20|   160|     954|  null|    null|2020-08-06 09:30:...|       T|


In [4]:
# Calculate the 30 min moving avg using the spark temp view
mov_avg_df = spark.sql("""
    select *,
    AVG(bid_pr) OVER (PARTITION BY (symbol) ORDER BY CAST(event_tm AS timestamp) 
    RANGE BETWEEN INTERVAL 30 MINUTES PRECEDING AND CURRENT ROW) as mov_avg_pr    
    from tmp_trade_moving_avg""")

# Save Temp View as a Hive Table
#mov_avg_df.write.saveAsTable("moving_avg2")

# Save temp view as temp view for easy access
mov_avg_df.createOrReplaceTempView('moving_avg2')
mov_avg_df.show(5)

+----------+------+--------+--------------------+------------+------+--------+------+--------+--------------------+--------+----------+
|  trade_dt|Symbol|exchange|            event_tm|event_seq_nb|bid_pr|bid_size|ask_pr|ask_size|    max_arrival_time|rec_type|mov_avg_pr|
+----------+------+--------+--------------------+------------+------+--------+------+--------+--------------------+--------+----------+
|2020-08-06|  SYMA|    NYSE|2020-08-06 10:49:...|          10|    74|      27|  null|    null|2020-08-06 09:30:...|       T|   74.0000|
|2020-08-06|  SYMA|    NYSE|2020-08-06 12:00:...|          20|    76|     315|  null|    null|2020-08-06 09:30:...|       T|   76.0000|
|2020-08-06|  SYMA|    NYSE|2020-08-06 13:11:...|          30|    77|     755|  null|    null|2020-08-06 09:30:...|       T|   77.0000|
|2020-08-06|  SYMA|    NYSE|2020-08-06 14:27:...|          40|    77|      81|  null|    null|2020-08-06 09:30:...|       T|   77.0000|
|2020-08-06|  SYMA|    NYSE|2020-08-06 15:39:...

In [5]:
# Recall that our dataset is comprised of two dates ranging form 8/05 to 8/06. Thus, our previous date should be '2020-08-05'

prior_df = df_8_05.select('*')
prior_df.show(5)
df_8_05.show(5)

prior_df.createOrReplaceTempView('tmp_last_trade')


+----------+------+--------+--------------------+------------+------+--------+------+--------+--------------------+
|  trade_dt|Symbol|exchange|            event_tm|event_seq_nb|bid_pr|bid_size|ask_pr|ask_size|    max_arrival_time|
+----------+------+--------+--------------------+------------+------+--------+------+--------+--------------------+
|2020-08-05|  SYMC|    NYSE|2020-08-05 21:52:...|         100|   161|     312|  null|    null|2020-08-05 09:30:...|
|2020-08-05|  SYMC|    NYSE|2020-08-05 19:37:...|          80|   161|     713|  null|    null|2020-08-05 09:30:...|
|2020-08-05|  SYMB|    NYSE|2020-08-05 14:26:...|          40|    37|     370|  null|    null|2020-08-05 09:30:...|
|2020-08-05|  SYMC|    NYSE|2020-08-05 20:48:...|          90|   160|     692|  null|    null|2020-08-05 09:30:...|
|2020-08-05|  SYMA|    NYSE|2020-08-05 10:37:...|          10|    79|     912|  null|    null|2020-08-05 09:30:...|
+----------+------+--------+--------------------+------------+------+---

In [6]:
# Calculate last trade price using the spark temp view

last_pr_df = spark.sql("""select last.trade_dt, last.symbol, last.exchange, last.event_tm, 
last.event_seq_nb, last.last_pr from 
(select trade_dt, symbol, exchange, event_tm, event_seq_nb, bid_pr, AVG(bid_pr)
OVER (PARTITION BY (symbol) ORDER BY CAST(event_tm as timestamp)RANGE BETWEEN INTERVAL 30 MINUTES PRECEDING AND CURRENT ROW) 
as last_pr
FROM moving_avg2) last
""")

last_pr_df.createOrReplaceTempView("temp_last_trade")
last_pr_df.show(5)

+----------+------+--------+--------------------+------------+-------+
|  trade_dt|symbol|exchange|            event_tm|event_seq_nb|last_pr|
+----------+------+--------+--------------------+------------+-------+
|2020-08-06|  SYMA|    NYSE|2020-08-06 10:49:...|          10|74.0000|
|2020-08-06|  SYMA|    NYSE|2020-08-06 12:00:...|          20|76.0000|
|2020-08-06|  SYMA|    NYSE|2020-08-06 13:11:...|          30|77.0000|
|2020-08-06|  SYMA|    NYSE|2020-08-06 14:27:...|          40|77.0000|
|2020-08-06|  SYMA|    NYSE|2020-08-06 15:39:...|          50|76.0000|
+----------+------+--------+--------------------+------------+-------+
only showing top 5 rows

In [7]:
# Load quote data into dataframes for the newest (8/06) and prior dates (8/05)
quote_df_new= spark.read.parquet("wasbs://guidedcapstone-2021-08-09t17-05-44-969z@guidedcapstonhdistorage.blob.core.windows.net/HdiNotebooks/trade3/quote_dt={}".format("2020-08-06"))
quote_df_prior = spark.read.parquet("wasbs://guidedcapstone-2021-08-09t17-05-44-969z@guidedcapstonhdistorage.blob.core.windows.net/HdiNotebooks/trade3/quote_dt={}".format("2020-08-05"))

quote_df_new = quote_df_new.select('*')
quote_df_new = quote_df_new.withColumn('rec_type', when(quote_df_new.exchange=='NASDAQ', 'Q'))
quote_df_new = quote_df_new.withColumn('mov_avg_pr', when(quote_df_new.exchange=='NASDAQ', 'null'))
quote_df_new.createOrReplaceTempView('quotes')
quote_df_new.show(5)

+----------+------+--------+--------------------+------------+------+--------+------+--------+--------------------+--------+----------+
|  trade_dt|Symbol|exchange|            event_tm|event_seq_nb|bid_pr|bid_size|ask_pr|ask_size|    max_arrival_time|rec_type|mov_avg_pr|
+----------+------+--------+--------------------+------------+------+--------+------+--------+--------------------+--------+----------+
|2020-08-06|  SYMB|  NASDAQ|2020-08-06 20:07:...|          90|  null|    null|    36|     546|2020-08-06 09:30:...|       Q|      null|
|2020-08-06|  SYMC|  NASDAQ|2020-08-06 15:30:...|          50|  null|    null|   161|     652|2020-08-06 09:30:...|       Q|      null|
|2020-08-06|  SYMA|  NASDAQ|2020-08-06 19:09:...|          80|  null|    null|    79|     307|2020-08-06 09:30:...|       Q|      null|
|2020-08-06|  SYMA|  NASDAQ|2020-08-06 15:39:...|          50|  null|    null|    78|     291|2020-08-06 09:30:...|       Q|      null|
|2020-08-06|  SYMC|  NASDAQ|2020-08-06 10:44:...

In [8]:
union_df = spark.sql("""
    select *,
    last_value(bid_pr, true) OVER (PARTITION BY symbol, exchange ORDER BY event_tm ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as last_trade_pr
    from moving_avg2
    
    union all
    
    select *,
    last_value(ask_pr, true) OVER (PARTITION BY symbol, exchange ORDER BY event_tm ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as last_trade_pr
    from quotes
    """)

# To check whether or not the moving avg price is being carried through correctly
union_df.where(col('rec_type')=='Q').show(5)
union_df.where(col('rec_type')=='T').show(5)


union_df.createOrReplaceTempView('quote_union_update')
#union_df.show(5)

+----------+------+--------+--------------------+------------+------+--------+------+--------+--------------------+--------+----------+-------------+
|  trade_dt|Symbol|exchange|            event_tm|event_seq_nb|bid_pr|bid_size|ask_pr|ask_size|    max_arrival_time|rec_type|mov_avg_pr|last_trade_pr|
+----------+------+--------+--------------------+------------+------+--------+------+--------+--------------------+--------+----------+-------------+
|2020-08-06|  SYMC|  NASDAQ|2020-08-06 10:44:...|          10|  null|    null|   159|     336|2020-08-06 09:30:...|       Q|      null|          159|
|2020-08-06|  SYMC|  NASDAQ|2020-08-06 11:54:...|          20|  null|    null|   161|      89|2020-08-06 09:30:...|       Q|      null|          161|
|2020-08-06|  SYMC|  NASDAQ|2020-08-06 13:07:...|          30|  null|    null|   160|     668|2020-08-06 09:30:...|       Q|      null|          160|
|2020-08-06|  SYMC|  NASDAQ|2020-08-06 14:15:...|          40|  null|    null|   157|      62|2020-0

In [9]:
# Populate latest trade_pr and mov_avg_pr

quote_union_update = spark.sql("""
select trade_dt, symbol, exchange, event_tm, event_seq_nb, bid_pr, bid_size, ask_pr, ask_size, rec_type, mov_avg_pr,
last_value(last_trade_pr, true) OVER (PARTITION BY symbol, exchange ORDER BY event_tm ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as last_trade_pr,
last_value(mov_avg_pr, true) OVER (PARTITION BY symbol, exchange ORDER BY event_tm ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as last_mov_avg_pr
from quote_union_update
""")

quote_union_update.createOrReplaceTempView("quote_union_update")
quote_union_update.show(5)

+----------+------+--------+--------------------+------------+------+--------+------+--------+--------+----------+-------------+---------------+
|  trade_dt|symbol|exchange|            event_tm|event_seq_nb|bid_pr|bid_size|ask_pr|ask_size|rec_type|mov_avg_pr|last_trade_pr|last_mov_avg_pr|
+----------+------+--------+--------------------+------------+------+--------+------+--------+--------+----------+-------------+---------------+
|2020-08-06|  SYMC|    NYSE|2020-08-06 10:42:...|          10|   157|     393|  null|    null|       T|  157.0000|          157|       157.0000|
|2020-08-06|  SYMC|    NYSE|2020-08-06 11:52:...|          20|   160|     954|  null|    null|       T|  160.0000|          160|       160.0000|
|2020-08-06|  SYMC|    NYSE|2020-08-06 13:01:...|          30|   158|     879|  null|    null|       T|  158.0000|          158|       158.0000|
|2020-08-06|  SYMC|    NYSE|2020-08-06 14:15:...|          40|   161|     232|  null|    null|       T|  161.0000|          161|  

In [10]:
spark.sql("""
select * from quote_union_update
""").show(5)

spark.sql("""
select * from temp_last_trade
""").show(5)

+----------+------+--------+--------------------+------------+------+--------+------+--------+--------+----------+-------------+---------------+
|  trade_dt|symbol|exchange|            event_tm|event_seq_nb|bid_pr|bid_size|ask_pr|ask_size|rec_type|mov_avg_pr|last_trade_pr|last_mov_avg_pr|
+----------+------+--------+--------------------+------------+------+--------+------+--------+--------+----------+-------------+---------------+
|2020-08-06|  SYMC|    NYSE|2020-08-06 10:42:...|          10|   157|     393|  null|    null|       T|  157.0000|          157|       157.0000|
|2020-08-06|  SYMC|    NYSE|2020-08-06 11:52:...|          20|   160|     954|  null|    null|       T|  160.0000|          160|       160.0000|
|2020-08-06|  SYMC|    NYSE|2020-08-06 13:01:...|          30|   158|     879|  null|    null|       T|  158.0000|          158|       158.0000|
|2020-08-06|  SYMC|    NYSE|2020-08-06 14:15:...|          40|   161|     232|  null|    null|       T|  161.0000|          161|  

In [11]:
# Filter for quote records
test=spark.sql("""select /* + BROADCAST(t) */
    q.trade_dt, q.symbol,q.event_tm,q.event_seq_nb, q.exchange, q.last_trade_pr, q.last_mov_avg_pr,
    t.last_pr as close_pr
    from quote_union_update as q
    left outer join temp_last_trade as t on
    (q.symbol = t.symbol and q.event_tm = t.event_tm)""")
test.show(5)



+----------+------+--------------------+------------+--------+-------------+---------------+--------+
|  trade_dt|symbol|            event_tm|event_seq_nb|exchange|last_trade_pr|last_mov_avg_pr|close_pr|
+----------+------+--------------------+------------+--------+-------------+---------------+--------+
|2020-08-06|  SYMC|2020-08-06 10:42:...|          10|    NYSE|          157|       157.0000|157.0000|
|2020-08-06|  SYMC|2020-08-06 11:52:...|          20|    NYSE|          160|       160.0000|160.0000|
|2020-08-06|  SYMC|2020-08-06 13:01:...|          30|    NYSE|          158|       158.0000|158.0000|
|2020-08-06|  SYMC|2020-08-06 14:15:...|          40|    NYSE|          161|       161.0000|161.0000|
|2020-08-06|  SYMC|2020-08-06 15:18:...|          50|    NYSE|          158|       158.0000|158.0000|
+----------+------+--------------------+------------+--------+-------------+---------------+--------+
only showing top 5 rows

In [12]:
quote_final = spark.sql("""
select trade_dt, symbol, event_tm, event_seq_nb, exchange,
bid_pr, bid_size, ask_pr, ask_size, last_trade_pr, last_mov_avg_pr,
bid_pr - close_pr as bid_pr_mv, ask_pr - close_pr as ask_pr_mv
from(
    
    select /* + BROADCAST(t) */
    q.trade_dt, q.symbol,q.event_tm,q.event_seq_nb, q.exchange, q.last_trade_pr, q.last_mov_avg_pr,q.bid_pr, q.bid_size, q.ask_pr, q.ask_size,
    t.last_pr as close_pr
    from quote_union_update as q
    left outer join temp_last_trade as t on
    (q.symbol = t.symbol and q.event_tm = t.event_tm))
""")

quote_final.show(5)

+----------+------+--------------------+------------+--------+------+--------+------+--------+-------------+---------------+---------+---------+
|  trade_dt|symbol|            event_tm|event_seq_nb|exchange|bid_pr|bid_size|ask_pr|ask_size|last_trade_pr|last_mov_avg_pr|bid_pr_mv|ask_pr_mv|
+----------+------+--------------------+------------+--------+------+--------+------+--------+-------------+---------------+---------+---------+
|2020-08-06|  SYMC|2020-08-06 10:42:...|          10|    NYSE|   157|     393|  null|    null|          157|       157.0000|   0.0000|     null|
|2020-08-06|  SYMC|2020-08-06 11:52:...|          20|    NYSE|   160|     954|  null|    null|          160|       160.0000|   0.0000|     null|
|2020-08-06|  SYMC|2020-08-06 13:01:...|          30|    NYSE|   158|     879|  null|    null|          158|       158.0000|   0.0000|     null|
|2020-08-06|  SYMC|2020-08-06 14:15:...|          40|    NYSE|   161|     232|  null|    null|          161|       161.0000|   0.0

In [13]:
trade_date1='2020-08-06'
quote_final.write.parquet("wasbs://guidedcapstone-2021-08-09t17-05-44-969z@guidedcapstonhdistorage.blob.core.windows.net/HdiNotebooks/final_trade/trade_dt={}".format(trade_date1))